In [1]:
import os, shutil
import tensorflow as tf
from tensorflow import keras
from keras.preprocessing.image import ImageDataGenerator
from keras import layers
from keras import models
from keras import optimizers
from tensorflow.keras.optimizers import Adam
from keras.preprocessing import image
import numpy as np
from sklearn.metrics import confusion_matrix
import matplotlib.pyplot as plt
import mlflow
import mlflow.tensorflow
from PIL import Image
import re

In [12]:
n_neurons=64
drop_out=0.1
labels=['','']
model = keras.applications.MobileNet(
weights='imagenet',
input_shape=(300, 300, 3),
include_top=False
)
model.trainable = False
inputs = keras.Input(shape=(300, 300, 3))
x = model(inputs, training=False)
#pooling layer 
x = keras.layers.GlobalAveragePooling2D()(x)
x = keras.layers.Dense(n_neurons, activation = 'softmax')(x)
x = keras.layers.Dropout(drop_out, input_shape=(n_neurons,))(x)
#final dense layer
outputs = keras.layers.Dense(2, activation = 'softmax')(x)
model = keras.Model(inputs,outputs)
model.compile(optimizer='adam', loss = 'categorical_crossentropy', metrics=['accuracy', 
                tf.keras.metrics.Precision(),
                tf.keras.metrics.Recall()])

In [13]:
json_topology=model.to_json()
with open("sample.json", "w") as outfile:
    outfile.write(json_topology)

In [14]:
checkpoint_path=r'C:\Users\USUARIO\Desktop\Tesis\gitrepo\cocoa_dl_android\model_fito.h5'
model.load_weights(checkpoint_path)

In [15]:
Model_json = r"C:\Users\USUARIO\Desktop\Tesis\gitrepo\cocoa_dl_android\sample.json"
Model_weights = r"C:\Users\USUARIO\Desktop\Tesis\gitrepo\cocoa_dl_android\model_fito.h5"

model_json = open(Model_json, 'r')
loaded_model_json = model_json.read()
model_json.close()
model = tf.keras.models.model_from_json(loaded_model_json)
model.load_weights(Model_weights)

In [1]:
import base64

In [9]:
string="""
...
dfv/AXoGbnIvhEHAAAAAAElFTkSuQmCC
"""

In [19]:
import base64
with open("220128-chihuahua-mb-0853-a252ab.jpg", "rb") as img_file:
    my_string = base64.b64encode(img_file.read())

my_string.is


True

In [2]:
print(my_string)

b'...'


In [2]:
text_file = open("C:/Users/USUARIO/Desktop/Tesis/gitrepo/cocoa_dl_android/img_to_b64.txt", "w")
 
#write string to file
text_file.write(str(my_string))
 
#close file
text_file.close()

In [4]:
from PIL import Image
import numpy as np
import base64
from io import BytesIO

In [10]:
img_string=str(string)
im_bytes = base64.b64decode(img_string)   # im_bytes is a binary image
im_file = BytesIO(im_bytes)  # convert image to file-like object

In [16]:
im_file

In [11]:
img = Image.open(im_file)   # img is now PIL Image object
img = img.resize((300, 300))

In [12]:
img.show()

In [5]:
type(img)

PIL.Image.Image

In [1]:
import tensorflow as tf

In [10]:
try:
    x = tf.keras.utils.img_to_array(img)
    # x = np.true_divide(x, 255)
    x = np.expand_dims(x, axis=0)
except Exception as e:
    print(f"Exception preprocessing img: {e}" )

In [13]:
MODEL_PATH=r"C:\Users\USUARIO\Desktop\Tesis\gitrepo\cocoa_dl_android\FlaskWebService2\src"
# Load Model
#==============================================================================

Model_json = MODEL_PATH+"/model.json"
Model_weights = MODEL_PATH+"/model.h5"

model_json = open(Model_json, 'r')
loaded_model_json = model_json.read()
model_json.close()
model = tf.keras.models.model_from_json(loaded_model_json)
model.load_weights(Model_weights)

In [14]:
from flask import Flask, jsonify

ModuleNotFoundError: No module named 'click'

In [15]:
try:
    preds = model.predict(x)
    preds=preds.tolist()[0]
    
    if preds[0]==1:
        class_pred="Cat"
    elif preds[1]==1:
        class_pred="Dog"
    print(str(class_pred))
except Exception as e:
    print(f"Exception making predictions img: {e}" )

1/1 [==============================] - 6s 6s/step
Exception making predictions img: name 'class_pred' is not defined


In [17]:
type(preds)

list

In [18]:
np.argmax(np.array(preds))

0